In [1]:
##########################################
# Comparision of PCA and PCA incremental PCA
# Author: Tomasz Hachaj
# e-mail: thachaj@agh.edu.pl
# 2025
##########################################

import numpy as np
path_to_data = 'data'
how_many_images = 3500
T = np.load(path_to_data + "//CREDO_T_st_" + str(how_many_images) + ".npy").transpose()

path_to_resultspartial = 'pca.res_CREDO'

batch_size=1000
start = 0
end = 0

import pickle
from sklearn.decomposition import IncrementalPCA
#transformer = IncrementalPCA(n_components=T.shape[1], batch_size=batch_size)
# n_components must be less or equal to the batch number of samples 1000 for the first partial_fit call
transformer = IncrementalPCA(n_components=1000, batch_size=batch_size)

a = 0

while end < T.shape[0]:
    end = start + batch_size
    if end > T.shape[0]:
        end = T.shape[0]
    bb = T[start:end:]
    transformer.partial_fit(bb)

    my_path = path_to_resultspartial + '/pickle' + str(start) + '-' + str(end) + '.pkl'
    with open(my_path, 'wb') as outp:
        pickle.dump(transformer, outp, pickle.HIGHEST_PROTOCOL)
    transformer = None

    with open(my_path, 'rb') as inp:
        transformer = pickle.load(inp)

    start = end
    print(str(a) + " of " + str(int(T.shape[0] / batch_size)))
    a = a + 1

0 of 3
1 of 3
2 of 3
3 of 3


In [2]:
def correct_vector(v1):
    v_help = np.copy(v1)
    max_val = np.max(v_help)
    min_val = np.min(v_help)
    if np.abs(min_val) > np.abs(max_val) and min_val < 0:
        v_help *= -1
    return v_help

import math
def planar_angle(v1, v2):
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    if math.isnan(angle):
        angle = 0
    return angle

# path to results computed with PCA
# Author: Tomasz Hachaj
# e-mail: tomekhachaj@o2.pl
# 2023
##########################################
# Comparison of coordinate frames obtained with basic PCA to coordinate frames obtained with Incremental PCA
# for a different number of data used when approximating PCA

import numpy as np
import pickle

def correct_vector(v1):
    v_help = np.copy(v1)
    max_val = np.max(v_help)
    min_val = np.min(v_help)
    if np.abs(min_val) > np.abs(max_val) and min_val < 0:
        v_help *= -1
    return v_help

import math
def planar_angle(v1, v2):
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    if math.isnan(angle):
        angle = 0
    return angle


# path to results computed with PCA
path_to_results = "pca.res_CREDO//"
# path to results computed with Incremental PCA
# path to store results
file = open('axes_similarityPCA.txt', 'w')

#start = 560000
#end =  570000
start = 0
end =  1000
how_many_images = 3500

v_correct = np.load(path_to_results + "//CREDO_v_st_" + str(how_many_images) + ".npy")
w = np.load(path_to_results + "//CREDO_w_st_" + str(how_many_images) + ".npy")
v_correct_scalled = w / np.sum(w)

transformer = None

while end < T.shape[0]:
    end = start + batch_size
    if end > T.shape[0]:
        end = T.shape[0]
    my_path = path_to_resultspartial + '//pickle' + str(start) + '-' + str(end) + '.pkl'
    with open(my_path, 'rb') as inp:
        transformer = pickle.load(inp)
    cumulatice_sum = 0
    sum_angle_res = 0
    # calculation of coordinate frames weighted distance (cfd)
    #for a in range(v_correct.shape[1]):
    for a in range(min(v_correct.shape[1], transformer.components_.shape[0])):
        v_help1 = correct_vector(v_correct[:, a])
        #print(transformer.components_.shape)
        v_help2 = correct_vector(transformer.components_[a, :])
        angle_res = planar_angle(v_help1, v_help2)
        average_eigen = (v_correct_scalled[a] + transformer.explained_variance_ratio_[a]) / 2
        cumulatice_sum += average_eigen
        sum_angle_res += average_eigen * angle_res
    print(str(start) + "," + str(end) + "," + str(sum_angle_res))
    start = end
    #file.write(str(start) + "," + str(end) + "," + str(sum_angle_res) + '\n')

0,1000,0.12416908228499189
1000,2000,0.10729322801072803
2000,3000,0.06295773816030653
3000,3500,0.0035763974363098652
